In [ ]:
import json
import matplotlib.pyplot as plt
from jiwer import wer, cer
import re
import numpy as np
from collections import defaultdict

In [ ]:
# Chargement des données
with open("transcriptions_ordonnées.json", encoding="utf-8") as f:
    data = json.load(f)

In [1]:
# Fonctions utilitaires
def concat_segments(d, prefix):
    textes = []
    i = 1
    while f"{prefix} ({i})" in d:
        textes.append(d[f"{prefix} ({i})"])
        i += 1
    return " ".join(textes)

def clean(t):
    return re.sub(r"[^\w\sąćęłńóśźżĄĆĘŁŃÓŚŹŻ-]", "", t.lower().strip())

# Initialisation des listes
wer_auto_vs_man, wer_auto_vs_corr = [], []
cer_auto_vs_man, cer_auto_vs_corr = [], []
ids = []

#Traitement des exemples
for exemple in data:
    apprenant_id = exemple.get("numéro d'apprenant", "inconnu")
    ids.append(apprenant_id)

    texte_auto = exemple.get("transcription automatique", 
                             exemple.get("transcription_automatique_non_pl", ""))
    texte_man = concat_segments(exemple, "transcription manuelle")
    texte_corr = concat_segments(exemple, "version correcte en polonais")

    texte_auto = clean(texte_auto)
    texte_man = clean(texte_man)
    texte_corr = clean(texte_corr)

    wer_auto_vs_man.append(wer(texte_man, texte_auto))
    wer_auto_vs_corr.append(wer(texte_corr, texte_auto))
    cer_auto_vs_man.append(cer(texte_man, texte_auto))
    cer_auto_vs_corr.append(cer(texte_corr, texte_auto))

# Dictionnaires pour stocker les scores par pays
scores_par_pays = defaultdict(lambda: {
    "wer_man": [],
    "wer_corr": [],
    "cer_man": [],
    "cer_corr": []
})

# Remplir les scores
for i, exemple in enumerate(data):
    pays = exemple.get("pays", "INCONNU")
    scores_par_pays[pays]["wer_man"].append(wer_auto_vs_man[i])
    scores_par_pays[pays]["wer_corr"].append(wer_auto_vs_corr[i])
    scores_par_pays[pays]["cer_man"].append(cer_auto_vs_man[i])
    scores_par_pays[pays]["cer_corr"].append(cer_auto_vs_corr[i])

# Affichage par pays
print("\n=== Statistiques globales par pays ===")
for pays in sorted(scores_par_pays.keys()):
    s = scores_par_pays[pays]
    print(f"\nPays : {pays}")
    print(f"WER moyen (Auto vs Manuel)  : {np.mean(s['wer_man']):.3f}")
    print(f"WER moyen (Auto vs Correct) : {np.mean(s['wer_corr']):.3f}")
    print(f"CER moyen (Auto vs Manuel)  : {np.mean(s['cer_man']):.3f}")
    print(f"CER moyen (Auto vs Correct) : {np.mean(s['cer_corr']):.3f}")
    print(f"WER médian (Auto vs Manuel)  : {np.median(s['wer_man']):.3f}")
    print(f"WER médian (Auto vs Correct) : {np.median(s['wer_corr']):.3f}")
    print(f"CER médian (Auto vs Manuel)  : {np.median(s['cer_man']):.3f}")
    print(f"CER médian (Auto vs Correct) : {np.median(s['cer_corr']):.3f}")


# Moyennes et médianes globales 
print("\n=== Statistiques globales ===")
print(f"WER moyen (Auto vs Manuel)  : {np.mean(wer_auto_vs_man):.3f}")
print(f"WER moyen (Auto vs Correct) : {np.mean(wer_auto_vs_corr):.3f}")
print(f"CER moyen (Auto vs Manuel)  : {np.mean(cer_auto_vs_man):.3f}")
print(f"CER moyen (Auto vs Correct) : {np.mean(cer_auto_vs_corr):.3f}")

print(f"WER médian (Auto vs Manuel)  : {np.median(wer_auto_vs_man):.3f}")
print(f"WER médian (Auto vs Correct) : {np.median(wer_auto_vs_corr):.3f}")
print(f"CER médian (Auto vs Manuel)  : {np.median(cer_auto_vs_man):.3f}")
print(f"CER médian (Auto vs Correct) : {np.median(cer_auto_vs_corr):.3f}")

# WER & CER auto vs manuelle
plt.figure(figsize=(30, 6))
plt.plot(ids, wer_auto_vs_man, marker='o', label="WER", color='blue')
plt.plot(ids, cer_auto_vs_man, marker='x', label="CER", color='orange')
plt.xticks(rotation=90)
plt.ylim(0, 4)
plt.title("Auto vs Manuel : WER & CER")
plt.xlabel("ID Apprenant")
plt.ylabel("Taux d'erreur")
plt.legend()
plt.tight_layout()
plt.savefig("images/wer_cer_auto_vs_man.png")
plt.close()

# WER & CER auto vs correct
plt.figure(figsize=(30, 6))
plt.plot(ids, wer_auto_vs_corr, marker='o', label="WER", color='green')
plt.plot(ids, cer_auto_vs_corr, marker='x', label="CER", color='red')
plt.xticks(rotation=90)
plt.ylim(0, 4)
plt.title("Auto vs Correct : WER & CER")
plt.xlabel("ID Apprenant")
plt.ylabel("Taux d'erreur")
plt.legend()
plt.tight_layout()
plt.savefig("images/wer_cer_auto_vs_corr.png")
plt.close()

# Comparaison globale des 4 courbes
plt.figure(figsize=(30, 6))
plt.plot(ids, wer_auto_vs_man, marker='o', label="WER Auto vs Man", color='blue')
plt.plot(ids, cer_auto_vs_man, marker='x', label="CER Auto vs Man", color='orange')
plt.plot(ids, wer_auto_vs_corr, marker='o', label="WER Auto vs Corr", color='green')
plt.plot(ids, cer_auto_vs_corr, marker='x', label="CER Auto vs Corr", color='red')
plt.xticks(rotation=90)
plt.ylim(0, 4)
plt.title("Comparaison globale : WER & CER")
plt.xlabel("ID Apprenant")
plt.ylabel("Taux d'erreur")
plt.legend()
plt.tight_layout()
plt.savefig("images/wer_cer_comparison.png")
plt.close()



=== Statistiques globales par pays ===

Pays : FR
WER moyen (Auto vs Manuel)  : 0.728
WER moyen (Auto vs Correct) : 0.743
CER moyen (Auto vs Manuel)  : 0.424
CER moyen (Auto vs Correct) : 0.462
WER médian (Auto vs Manuel)  : 0.500
WER médian (Auto vs Correct) : 0.535
CER médian (Auto vs Manuel)  : 0.263
CER médian (Auto vs Correct) : 0.286

Pays : GE
WER moyen (Auto vs Manuel)  : 0.652
WER moyen (Auto vs Correct) : 0.755
CER moyen (Auto vs Manuel)  : 0.384
CER moyen (Auto vs Correct) : 0.435
WER médian (Auto vs Manuel)  : 0.406
WER médian (Auto vs Correct) : 0.463
CER médian (Auto vs Manuel)  : 0.163
CER médian (Auto vs Correct) : 0.188

Pays : IT
WER moyen (Auto vs Manuel)  : 0.917
WER moyen (Auto vs Correct) : 0.896
CER moyen (Auto vs Manuel)  : 0.615
CER moyen (Auto vs Correct) : 0.622
WER médian (Auto vs Manuel)  : 0.522
WER médian (Auto vs Correct) : 0.475
CER médian (Auto vs Manuel)  : 0.238
CER médian (Auto vs Correct) : 0.262

Pays : NL
WER moyen (Auto vs Manuel)  : 0.486
WER 

In [5]:
# Corpus polonais

# Chargement des données depuis le fichier JSON
with open("corpus_polonais_natif.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Listes pour le calcul global
references = []
hypotheses = []

print("=== Résultats par exemple ===\n")

for exemple in data:
    ref = exemple["transcription manuelle"]
    hyp = exemple["transcription automatique"]

    # Calcul du WER et du CER
    wer_val = wer(ref, hyp)
    cer_val = cer(ref, hyp)

    print(f"{exemple['name']}")
    print(f"WER : {wer_val:.3f}")
    print(f"CER : {cer_val:.3f}\n")

    references.append(ref)
    hypotheses.append(hyp)

# Calcul global (toutes les transcriptions concaténées)
ref_global = " ".join(references)
hyp_global = " ".join(hypotheses)

wer_global = wer(ref_global, hyp_global)
cer_global = cer(ref_global, hyp_global)

print("=== Résultats globaux ===")
print(f"WER global : {wer_global:.3f}")
print(f"CER global : {cer_global:.3f}")


=== Résultats par exemple ===

Monika
WER : 0.075
CER : 0.043

Ola
WER : 0.180
CER : 0.064

Magda
WER : 0.167
CER : 0.081

Klaudia
WER : 0.139
CER : 0.072

=== Résultats globaux ===
WER global : 0.137
CER global : 0.064
